In [1]:
import pandas as pd
import nltk
from nltk.collocations import *
from scipy.stats import spearmanr

In [53]:
df = pd.read_csv('hw_data.csv', header=None) # corpus
#newdf = df[df.columns[1:5]]
rows = list(map(list,df.values)) # list of rows in corpus

### Объем корпуса

In [80]:
len(rows)

1020

### Объем корпуса в токенах

In [85]:
tokens = sum([len(t) for t in rows])
tokens

5100

In [156]:
bigram_measures = nltk.collocations.BigramAssocMeasures()
finder = BigramCollocationFinder.from_documents(rows)
finder.apply_freq_filter(5)

### Золотой стандарт

In [124]:
golden_standard = [
    ('ОСВОБОДИТЬ', 'ЗАЛОГ'),
    ('МЕРА', 'ПРЕСЕЧЕНИЕ'),
    ('ПРИНЯТЬ', 'РЕШЕНИЕ'),
    ('УДОВЛЕТВОРИТЬ', 'ИСК'),
    ('ПРИЗНАТЬ', 'ВИНОВНАЯ'),
    ('ВЫНЕСТИ', 'РЕШЕНИЕ'),
    ('ОТКАЗАТЬ', 'УДОВЛЕТВОРЕНИЕ'),
    ('НАЛОЖИТЬ', 'АРЕСТ'),
    ('ОТКЛОНИТЬ', 'ИСК'),
    ('УДОВЛЕТВОРИТЬ', 'ХОДАТАЙСТВО')]

### Likelihood Ratio

### Chi-square Top-10

In [159]:
chi_score = finder.score_ngrams(bigram_measures.chi_sq)
chi = [x[0] for i,x in enumerate(chi_score[:10])]
chi

[('ОСВОБОДИТЬ ', 'ЗАЛОГ '),
 ('МЕРА ', 'ПРЕСЕЧЕНИЕ '),
 ('ЛИШЕНИЕ ', 'СВОБОДА '),
 ('ПРОЙТИ ', 'ПРЕНИЕ '),
 ('ВЫДАТЬ ', 'САНКЦИЯ '),
 ('ИЗБРАТЬ ', 'МЕРА '),
 ('САНКЦИЯ ', 'АРЕСТ '),
 ('ОСТАВИТЬ ', 'СИЛА '),
 ('НАЛОЖИТЬ ', 'АРЕСТ '),
 ('УДОВЛЕТВОРИТЬ ', 'ХОДАТАЙСТВО ')]

### В ЗС не вошли:
- ('ПРОЙТИ ', 'ПРЕНИЕ '),
- ('ВЫДАТЬ ', 'САНКЦИЯ '),
- ('ИЗБРАТЬ ', 'МЕРА '),
- ('ЛИШЕНИЕ ', 'СВОБОДА ')
- ('САНКЦИЯ ', 'АРЕСТ '),
- ('ОСТАВИТЬ ', 'СИЛА ')

### В ЗС можно было бы включить:
- ('ИЗБРАТЬ ', 'МЕРА ')

### PMI

In [160]:
#pmi = finder.nbest(bigram_measures.pmi, 10)
pmi_score = finder.score_ngrams(bigram_measures.pmi)
pmi = [x[0] for i,x in enumerate(pmi_score[:10])]

### PMI Top-10

In [165]:
pmi

[('ОСВОБОДИТЬ ', 'ЗАЛОГ '),
 ('ЛИШЕНИЕ ', 'СВОБОДА '),
 ('ПРОЙТИ ', 'ПРЕНИЕ '),
 ('МЕРА ', 'ПРЕСЕЧЕНИЕ '),
 ('ИЗБРАТЬ ', 'МЕРА '),
 ('ВЫДАТЬ ', 'САНКЦИЯ '),
 ('ОСТАВИТЬ ', 'СИЛА '),
 ('АРЕСТОВАТЬ ', 'АКЦИЯ '),
 ('СОГЛАСИТЬСЯ ', 'ДОВОД '),
 ('НАЧАТЬСЯ ', 'РАССМОТРЕНИЕ ')]

### В ЗС не попали:
- ('ЛИШЕНИЕ ', 'СВОБОДА ')
- ('ПРОЙТИ ', 'ПРЕНИЕ ')
- ('ИЗБРАТЬ ', 'МЕРА ')
- ('ВЫДАТЬ ', 'САНКЦИЯ '),
- ('ОСТАВИТЬ ', 'СИЛА '),
- ('АРЕСТОВАТЬ ', 'АКЦИЯ '),
- ('СОГЛАСИТЬСЯ ', 'ДОВОД '),
- ('НАЧАТЬСЯ ', 'РАССМОТРЕНИЕ ')


### В ЗС можно было бы включить:
- ('НАЧАТЬСЯ ', 'РАССМОТРЕНИЕ ')
- ('ИЗБРАТЬ ', 'МЕРА ')
- ('ЛИШЕНИЕ ', 'СВОБОДА ')

### Корреляция между результатами двух метрик

In [162]:
print('%0.1f' % spearman_correlation(ranks_from_sequence(chi), ranks_from_sequence(pmi)))

0.8


### PMI и ЗС

In [163]:
spearman_correlation(ranks_from_sequence(golden_standard), ranks_from_sequence(pmi))

0.0

### Chi и ЗС

In [164]:
spearman_correlation(ranks_from_sequence(golden_standard), ranks_from_sequence(chi))

0.0

#### Очень странно и кажется, что всё очень плохо!